In [1]:
# from adain import *
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG19

#==================================================
_V19In = 0x100 #0xE0
_V19InSq = _V19In * _V19In
_V19InHl = _V19In >> 0x1
_V19Out = _V19In >> 0x3

# VGG19 was trained by Caffe which converted images from RGB to BGR,
# then zero-centered each color channel with respect to the ImageNet 
# dataset, without scaling.  
_V19Mn = np.array([103.939, 116.779, 123.68], dtype = np.float32) # BGR
_V19Lo = -_V19Mn 
_V19Hi = 255.0 - _V19Mn 
_tfV19Mn = tf.convert_to_tensor(_V19Mn)
_tfV19Lo = tf.convert_to_tensor(_V19Lo)
_tfV19Hi = tf.convert_to_tensor(_V19Hi)
_V19HW = (_V19In, _V19In)
_V19HWhlf = (_V19InHl, _V19InHl)
_V19HWC = (_V19In, _V19In, 0x3)
_V19XSC = (-0x1, _V19InSq, 0x3)
_V19NSC = (None, _V19InSq, 0x3)
_V19BHWC = (0x1, _V19In, _V19In, 0x3)
_V19XHWC = (-0x1, _V19In, _V19In, 0x3)
_V19NHWC = (None, _V19In, _V19In, 0x3)
_V19B4C1 = (_V19Out, _V19Out, 0x200)
_V19B4C1N = (None, _V19Out, _V19Out, 0x200)

_YIQMl = np.array([0.114, 0.587, 0.299], dtype = np.float32) # BGR
_YIQBa = np.dot(_YIQMl, _V19Mn)
_YIQAx = (0x3, 0x0)
_ADA_TKN = "in_ada"
#==================================================


vgg = VGG19(include_top = False, weights = "imagenet", input_shape = _V19HWC)
############### Build the map for signature_def ################
mdl_c = keras.Model(
            inputs = vgg.input,
            outputs = [
                    vgg.get_layer("block1_conv1").output,
                    vgg.get_layer("block2_conv1").output,
                    vgg.get_layer("block3_conv1").output,
                    vgg.get_layer("block4_conv1").output,
                ],
            name = "vgg19"
        )
mdl_s = keras.Model(
            inputs = vgg.input,
            outputs = [
                    vgg.get_layer("block1_conv1").output,
                    vgg.get_layer("block2_conv1").output,
                    vgg.get_layer("block3_conv1").output,
                    vgg.get_layer("block4_conv1").output,
                ],
            name = "vgg19"
        )

In [2]:
def decoder(input_t, input_c, input_s):
    # Decoder
    _fd0 = input_t
#     _fd0 = keras.Input(shape = _V19B4C1, name = _ADA_TKN)
    _d1_1 = keras.layers.Conv2DTranspose(
                0x100,
                0x3,
                padding = "same", 
                activation = "relu"
    )
    _fd1_1 = _d1_1(_fd0)
    _d1_u = keras.layers.UpSampling2D(0x2)
    _fd1 = _d1_u(_fd1_1)
    _d2_1 = keras.layers.Conv2DTranspose(
            0x100,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd2_1 = _d2_1(_fd1)
    _d2_2 = keras.layers.Conv2DTranspose(
            0x100,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd2_2 = _d2_2(_fd2_1)
    _d2_3 = keras.layers.Conv2DTranspose(
            0x100,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd2_3 = _d2_3(_fd2_2)
    _d2_4 = keras.layers.Conv2DTranspose(
            0x80,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd2_4 = _d2_4(_fd2_3)
    _d2_u = keras.layers.UpSampling2D(0x2)
    _fd2 = _d2_u(_fd2_4)
    _d3_1 = keras.layers.Conv2DTranspose(
            0x80,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd3_1 = _d3_1(_fd2)
    _d3_2 = keras.layers.Conv2DTranspose(
            0x40,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd3_2 = _d3_2(_fd3_1)
    _d3_u = keras.layers.UpSampling2D(0x2)
    _fd3 = _d3_u(_fd3_2)
    _d4_1 = keras.layers.Conv2DTranspose(
            0x40,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd4_1 = _d4_1(_fd3)
    _d4_2 = keras.layers.Conv2DTranspose(
            0x3,
            0x3,
            padding = "same", 
            activation = "relu"
    )
    _fd4_2 = _d4_2(_fd4_1)
    decoder = keras.Model(inputs=[input_c], outputs = _fd4_2, name = "decoder")
    
    return decoder


In [3]:
input_c = mdl_c.input
input_s = mdl_s.input

#Encoder
ec = mdl_c.output[-1]
es = mdl_s.output[-1]

# AdaIN
eps = 1.0e-06
c0 = ec
s0 = es
uc, vc = tf.nn.moments(c0, axes = (0x1, 0x2), keepdims = True)
us, vs = tf.nn.moments(s0, axes = (0x1, 0x2), keepdims = True)
sc, ss = tf.sqrt(vc + eps), tf.sqrt(vs + eps)
nc =  ss * (c0 - uc) / sc + us

# Decoder
_decoder = decoder(nc, input_c, input_s)
out = _decoder.output[-1]
# Clip
output = tf.clip_by_value(out, _tfV19Lo, _tfV19Hi)

In [4]:
# load checkpoint
ckp = tf.train.latest_checkpoint("./mdl_L/")
if ckp : 
    print('load checkpoint...')
    _decoder.load_weights(ckp)
    

W0827 17:27:49.057454 140488729929472 base.py:270] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fc58a1c9a20> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fc589912e80>).
W0827 17:27:49.058351 140488729929472 base.py:270] Inconsistent references when loading the checkpoint into this object graph. Either the Trackable object references in the Python program have changed in an incompatible way, or the checkpoint was generated in an incompatible program.

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fc589899630> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f

load checkpoint...


In [5]:

tensor_info_c = tf.compat.v1.saved_model.build_tensor_info(input_c)
tensor_info_s = tf.compat.v1.saved_model.build_tensor_info(input_s)
tensor_info_output = tf.compat.v1.saved_model.build_tensor_info(output)
add_variable = False

# tensor to pass in base64
base64_img = tf.strings.as_string(output)
base64_img = tf.reshape(base64_img, [-1])
base64_img = tf.strings.reduce_join(base64_img, separator=',')
base64_img = tf.io.encode_base64(base64_img)
shape = tf.strings.as_string(output.shape[1:])
shape = tf.strings.reduce_join(shape, separator=',')
msg = tf.strings.join([shape, base64_img], separator='|')

# map method name to tensor info
map_dict = {}
map_dict[output.name] = tf.compat.v1.saved_model.build_tensor_info(msg)
##################################################################

# signature method name must be CLASSIFY_METHOD_NAME, PREDICT_METHOD_NAME or REGRESS_METHOD_NAME, 
# they have different request format. please refer to https://www.tensorflow.org/tfx/serving/signature_defs 
# and https://www.tensorflow.org/tfx/serving/api_rest/    
signature = (
    tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs={'input_c': tensor_info_c, 'input_s': tensor_info_s,},
        outputs=map_dict,
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME 
    )
)

version = 1
export_path = './model/{}'.format(version)
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_path)


builder.add_meta_graph_and_variables(
    tf.compat.v1.keras.backend.get_session(), [tf.compat.v1.saved_model.tag_constants.SERVING],
    signature_def_map={'style_transfer':signature,},
    strip_default_attrs=True
)
add_variable = True
builder.save()

W0827 17:27:49.436497 140488729929472 deprecation.py:323] From <ipython-input-5-3d41878ebe2a>:2: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


b'./model/1/saved_model.pb'